In [1]:
#importing necessary libraries
import ee
import folium
import geehydro
import numpy as np
import pandas as pd
import IPython.display as disp

In [2]:
#connecting to google earth engine API
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo()) #just a test if its working


Successfully saved authorization token.
NASADEM: NASA NASADEM Digital Elevation 30m


In [3]:
#function to collect the mean ndvi pixel values from the satellite per year per country
def ndvi_collection(country, year):
    location = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq("country_na", str(country)))
    dataset = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5").select("NDVI").filterDate(str(year)+"-01-01", str(year+1)+"-01-01")
    
    def setProperty(image):
        dict = image.reduceRegion(ee.Reducer.mean(), location, maxPixels=1e9)
        return image.set(dict)

    pixel_values = dataset.map(setProperty)
    aggregate_pixel_values = (pixel_values.aggregate_array("NDVI").getInfo())
    mean_pixel_value = np.mean(aggregate_pixel_values)

    return mean_pixel_value

In [4]:
#aggregating the values and saving to a list
ndvi_values = []
country_index = []
time_index = []

countries = ["Taiwan", "India", "Germany"] #add all countries of the original dataframe to this list
i = 0
while i < len(countries):
    year = 1990
    while year < 2014:
        result = ndvi_collection(country = countries[i], year = year)
        ndvi_values.append(result)
        country_index.append(countries[i])
        time_index.append(year)
        year += 1
    i += 1

In [5]:
#creating a dataframe with the values and corresponding years/countries
df_ndvi = pd.DataFrame()
df_ndvi["Country"] = country_index
df_ndvi["Year"] = time_index
df_ndvi["NDVI"] = ndvi_values
df_ndvi.to_csv("test_ndvi_df.csv") #saving the dataframe as a csv file to merge it with the other data later on

df_ndvi.head()

,Country,Year,NDVI
0,Taiwan,1990,1502.507333
1,Taiwan,1991,1475.033023
2,Taiwan,1992,1327.362254
3,Taiwan,1993,1395.165139
4,Taiwan,1994,1223.158209


In [6]:
df_ndvi.tail()

,Country,Year,NDVI
67,Germany,2009,1705.260078
68,Germany,2010,1378.547684
69,Germany,2011,1974.492369
70,Germany,2012,1677.682480
71,Germany,2013,1492.167126


In [7]:
#just a fun visualization of the data we collect in the function above - country and dates can be played with
coords = [52.15, 10.33]  # [lat, long]
Map = folium.Map(location=coords, zoom_start=6)

worldmap = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
location_filter = ee.Filter.eq("country_na","Taiwan")
country = worldmap.filter(location_filter)

dataset = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5")
dataset = dataset.select("NDVI")

date_filter = ee.Filter.date("2010-01-01", "2011-01-01")
dataset = dataset.filter(date_filter)

dataset = dataset.mean().clip(country)

visualisation_params = {
  "min": -1000,
  "max": 5000,
  "palette": ["ffffff", "ce7e45", "fcd163", "c6ca02", "22cc04", "99b718", "207401", "012e01"]
}

Map.setCenter(10.451526, 51.165691, 6)
Map.addLayer(country)
Map.centerObject(country, 6)
Map.addLayer(dataset, visualisation_params, "Default visualization")
Map